In [1]:
# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.books_5_small")
df3 = spark.sql("select * from default.home_and_kitchen_5_small")
df = df1.union(df2).union(df3)
print((df.count(), len(df.columns)))

(3487331, 12)

In [2]:
df.printSchema()

root
-- reviewID: integer (nullable = true)
-- overall: double (nullable = true)
-- vote: integer (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)

In [3]:
# Let's look at some quick summary statistics
df.describe().show()

+-------+-----------------+------------------+------------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+-------------------+
summary| reviewID| overall| vote|reviewTime| reviewerID| asin|reviewerName| reviewText| summary| unixReviewTime| label|
+-------+-----------------+------------------+------------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+-------------------+
 count| 3487331| 3487331| 3487331| 3487331| 3487331| 3487331| 3487072| 3487331| 3486921| 3487331| 3487331|
 mean|751050.9429308546| 4.320101533235589|1.9842919986660286| null| null|4.418121296960575E7| NaN| 6.845866966880342E7| Infinity|1.3923033604993618E9| 0.1795545074442317|
 stddev|554568.4455847011|1.1243846048419905|20.742122955150233| null| null|2.709716809221119E8| NaN| 7.404837276215339E8| NaN|1.0328806237719026E8|0.38381600871188076|
 min| 0| 1.0| 0|01 1, 1998|A0001528BGUBOEVR6T5U| 0000013765| | | | 843004800| 0|
 max| 2000417| 5.0| 7290|12 9, 2017| AZZZYAYJQSDOJ| B01HIZGKOE| ~~~|~~~~~ALL GREAT ...|~~~~(,,,___,,,)(=...| 1538265600| 1|
+-------+-----------------+------------------+------------------+----------+--------------------+-------------------+------------+--------------------+--------------------+--------------------+-------------------+

In [4]:
# The count of each overall rating

from pyspark.sql.functions import col
df.groupBy("overall").count().orderBy(col("overall").asc()).show()

+-------+-------+
overall| count|
+-------+-------+
 1.0| 179046|
 2.0| 153010|
 3.0| 291621|
 4.0| 612575|
 5.0|2251079|
+-------+-------+

In [5]:
# The most common product IDs
df.groupBy("asin").count().orderBy(col("count").desc()).show(10)

+----------+-----+
 asin|count|
+----------+-----+
0007420412|19655|
000711835X|18496|
0007548672|15921|
0007350899| 7890|
0007444117| 7830|
B000YGEVMI| 7356|
0007378033| 6336|
B0015TMHSI| 5846|
006017322X| 5161|
0007384289| 5095|
+----------+-----+
only showing top 10 rows

In [6]:
# For our intitial modeling efforts, we are not going to use the following features
drop_list = ['overall', 'summary', 'asin', 'reviewID', 'reviewerID', 'summary', 'unixReviewTime','reviewTime', 'image', 'style', 'verified', 'reviewerName']
df = df.select([column for column in df.columns if column not in drop_list])
df.show(5)
print((df.count(), len(df.columns)))

+----+--------------------+-----+
vote| reviewText|label|
+----+--------------------+-----+
 0|This game is a bi...| 0|
 0|I played it a whi...| 0|
 0| ok game.| 0|
 0|found the game a ...| 0|
 0|great game, I lov...| 0|
+----+--------------------+-----+
only showing top 5 rows

(3487331, 3)

In [7]:
df = df.na.drop(subset=["reviewText", "label"])
df.show(5)
print((df.count(), len(df.columns)))

+----+--------------------+-----+
vote| reviewText|label|
+----+--------------------+-----+
 0|This game is a bi...| 0|
 0|I played it a whi...| 0|
 0| ok game.| 0|
 0|found the game a ...| 0|
 0|great game, I lov...| 0|
+----+--------------------+-----+
only showing top 5 rows

(3487331, 3)

In [8]:
df.printSchema()

root
-- vote: integer (nullable = true)
-- reviewText: string (nullable = true)
-- label: integer (nullable = true)

In [9]:
df.groupBy("label").count().show()

+-----+-------+
label| count|
+-----+-------+
 1| 626166|
 0|2861165|
+-----+-------+

In [10]:
# set seed for reproducibility
(trainingData, testingData) = df.randomSplit([0.8, 0.2], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testingData.count()))

Training Dataset Count: 2790834
Test Dataset Count: 696497

In [11]:
# In Spark's MLLib, it's considered good practices to combine all the preprocessing steps into a pipeline.
# That way, you can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

# It is possible to run all of these steps one-by-one, outside of a Pipeline, if desired. But that's
# not how I am going to do it here.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer

# We'll tokenize the text using a simple RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")


# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")


# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)


pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors])


In [12]:
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(trainingData)
trainingDataTransformed = pipelineFit.transform(trainingData)
trainingDataTransformed.show(5)

+----+--------------------+-----+--------------------+--------------------+--------------------+
vote| reviewText|label| words| filtered| features|
+----+--------------------+-----+--------------------+--------------------+--------------------+
 0| The Bureau: XCO...| 0|[the, bureau, xco...|[bureau, xcom, de...|(10000,[0,2,4,5,6...|
 0| !!| 0| []| []| (10000,[],[])|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
+----+--------------------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

In [13]:
from pyspark.ml.classification import LogisticRegression

# More classification docs: https://spark.apache.org/docs/latest/ml-classification-regression.html

lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingDataTransformed)

In [14]:
# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

print("Training Accuracy:  " + str(trainingSummary.accuracy))
print("Training Precision: " + str(trainingSummary.precisionByLabel))
print("Training Recall:    " + str(trainingSummary.recallByLabel))
print("Training FMeasure:  " + str(trainingSummary.fMeasureByLabel()))
print("Training AUC:       " + str(trainingSummary.areaUnderROC))

Training Accuracy: 0.8415323161463563
Training Precision: [0.8509834769787344, 0.6854612999987449]
Training Recall: [0.9781071506100323, 0.21786722248262244]
Training FMeasure: [0.9101277009780026, 0.33064283000791556]
Training AUC: 0.8425716862866609

In [15]:
trainingSummary.roc.show()

+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
0.002035406463245...|0.039067346145024086|
0.004791504056180712| 0.07470804703160372|
0.008042913007339256| 0.10819662321861319|
0.011532368422546888| 0.1406460363208441|
0.015372123405034093| 0.17161349515821805|
0.019445993809044805| 0.20145602505160912|
0.023809887794942846| 0.2300280234958563|
 0.0284017822473246| 0.25760673361722497|
 0.03312951605809033| 0.28483440208231525|
 0.0380546755601408| 0.3108972505061183|
 0.04317376649345329| 0.3361782334227559|
 0.04855492692847119| 0.36031035273702794|
0.054020823169040506| 0.3840136427552781|
 0.05965007606567287| 0.4069889202477237|
 0.0652068230668334| 0.4280274849660427|
 0.07122088134846989| 0.44931137993278353|
 0.0773816985510614| 0.47007669063456764|
 0.08370805632223018| 0.4899304897629472|
 0.09022528804714106| 0.5092696937361004|
+--------------------+--------------------+
only showing top 20 rows

In [16]:
# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
for objective in objectiveHistory:
    print(objective)

0.4708578179738978
0.4551231601196833
0.42757050268205077
0.42163890531131315
0.4192786317846074
0.41248330986523357
0.4088110680229189
0.4041263004802277
0.40153445744264127
0.3993087134496857
0.39640158758446864
0.39406604207049367
0.39205905778879313
0.39154469380717905
0.39145074894526066
0.3914281529200909
0.3914208901934118
0.39141874869470905
0.39141648900991743
0.3914153210018971
0.39141473942863797

In [17]:
testingDataTransform = pipelineFit.transform(testingData)
testingDataTransform.show(5)

+----+--------------------+-----+--------------------+--------------------+--------------------+
vote| reviewText|label| words| filtered| features|
+----+--------------------+-----+--------------------+--------------------+--------------------+
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|
+----+--------------------+-----+--------------------+--------------------+--------------------+
only showing top 5 rows

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = lrModel.transform(testingDataTransform)
predictions.show(5)

evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
print('Test Area Under ROC', evaluator.evaluate(predictions))

+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
vote| reviewText|label| words| filtered| features| rawPrediction| probability|prediction|
+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|[2.27224730727430...|[0.90655234285074...| 0.0|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|[2.27224730727430...|[0.90655234285074...| 0.0|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|[2.27224730727430...|[0.90655234285074...| 0.0|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|[2.27224730727430...|[0.90655234285074...| 0.0|
 0| Bowden's Masonry...| 0|[bowden, s, mason...|[bowden, masonry,...|(10000,[67,121,48...|[2.27224730727430...|[0.90655234285074...| 0.0|
+----+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

Test Area Under ROC 0.8412175206914184

In [19]:
# Load in the tables
test_df = spark.sql("select * from default.reviews_test")
test_df.show(5)
print((test_df.count(), len(test_df.columns)))

+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+
reviewID|overall|vote|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime|
+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+
 1| 5| 0| true|01 15, 2016|AEKP8GO0B3YNF|1939289351| WADE POWERS|A very enjoyable ...| An Excellent Read| 1452816000|
 2| 5| 0| true| 07 7, 2014|APVIATPDFKY2O|1939289351| edfnewell| beautifully done1`| Five Stars| 1404691200|
 3| 5| 2| false| 07 5, 2014|A9X8E67M80P9A|1939289351|Rebecca D. Elswick|HIDING EZRA, by R...| A Love Story| 1404518400|
 4| 5| 0| true| 07 7, 2015|ABTZEFWA8MWYF|1939418046| Kay DeTellem|This was a real e...| Five Stars| 1436227200|
 5| 1| 0| false|04 25, 2017|A5C4231YJK1K7|1939346169| Bryan|What huge let dow...|Huh? What happened?| 1493078400|
+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+
only showing top 5 rows

(609970, 11)

In [20]:
test_df_Transform = pipelineFit.transform(test_df)
test_df_Transform.show(5)

+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+
reviewID|overall|vote|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime| words| filtered| features|
+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+
 1| 5| 0| true|01 15, 2016|AEKP8GO0B3YNF|1939289351| WADE POWERS|A very enjoyable ...| An Excellent Read| 1452816000|[a, very, enjoyab...|[enjoyable, read,...|(10000,[1,5,7,14,...|
 2| 5| 0| true| 07 7, 2014|APVIATPDFKY2O|1939289351| edfnewell| beautifully done1`| Five Stars| 1404691200|[beautifully, done1]|[beautifully, done1]| (10000,[842],[1.0])|
 3| 5| 2| false| 07 5, 2014|A9X8E67M80P9A|1939289351|Rebecca D. Elswick|HIDING EZRA, by R...| A Love Story| 1404518400|[hiding, ezra, by...|[hiding, ezra, ri...|(10000,[4,8,10,14...|
 4| 5| 0| true| 07 7, 2015|ABTZEFWA8MWYF|1939418046| Kay DeTellem|This was a real e...| Five Stars| 1436227200|[this, was, a, re...|[real, eye, opene...|(10000,[3,52,59,1...|
 5| 1| 0| false|04 25, 2017|A5C4231YJK1K7|1939346169| Bryan|What huge let dow...|Huh? What happened?| 1493078400|[what, huge, let,...|[huge, let, issue...|(10000,[5,37,82,2...|
+--------+-------+----+--------+-----------+-------------+----------+------------------+--------------------+-------------------+--------------+--------------------+--------------------+--------------------+
only showing top 5 rows

In [21]:
predictions = lrModel.transform(test_df_Transform)

reviewID,prediction
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,1.0
9,0.0
10,0.0


In [22]:
display(predictions.select('reviewID', 'prediction'))

reviewID,prediction
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,1.0
9,0.0
10,0.0
